# Depth Bias Analysis

In [ ]:
# Results

# apartment[[0, 5, 10, 15, 20, 25, 30, 35, 40]]
# Distance to Plane [m] to Incidence Angle (deg. 1 fit): poly([-0.00402863  0.00198255])
# Distance to Plane / Depth to Incidence Angle (deg. 1 fit): poly([-0.00127849  0.00051986])
# Distance to Plane / Depth to 1 / Incidence Angle Cosine (deg. 1 fit): poly([-8.41913328e-04  5.15523231e-20])

# eth[[0, 5, 10, 15, 20, 25, 30, 35]]
# Distance to Plane [m] to Incidence Angle (deg. 1 fit): poly([ 0.00193905 -0.00254346])
# Distance to Plane / Depth to Incidence Angle (deg. 1 fit): poly([ 0.00066698 -0.00070023])
# Distance to Plane / Depth to 1 / Incidence Angle Cosine (deg. 1 fit): poly([ 1.01255097e-04 -6.20008654e-21])

# gazebo_winter[[0, 5, 10, 15, 20, 25, 30]]
# Distance to Plane [m] to Incidence Angle (deg. 1 fit): poly([0.00015854 0.00017461])
# Distance to Plane / Depth to Incidence Angle (deg. 1 fit): poly([ 2.65784234e-04 -1.92807751e-05])
# Distance to Plane / Depth to 1 / Incidence Angle Cosine (deg. 1 fit): poly([ 2.50168305e-04 -1.53183907e-20])

# stairs[[0, 5, 10, 15, 20, 25, 30]]
# Distance to Plane [m] to Incidence Angle (deg. 1 fit): poly([-0.0016521   0.00064674])
# Distance to Plane / Depth to Incidence Angle (deg. 1 fit): poly([-2.13034350e-04 -4.96068873e-05])
# Distance to Plane / Depth to 1 / Incidence Angle Cosine (deg. 1 fit): poly([-2.57433793e-04  1.57632735e-20])

In [ ]:
# Imports

from __future__ import absolute_import, division, print_function
from data.asl_laser import Dataset, dataset_names
from depth_correction.depth_cloud import DepthCloud
from depth_correction.filters import filter_eigenvalue, filter_depth, filter_grid
from depth_correction.loss import reduce
from depth_correction.nearest_neighbors import nearest_neighbors
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import Polynomial
import torch
from timeit import default_timer as timer

In [ ]:
# Data preprocessing and analysis params

clouds = []
poses = []
# ds = Dataset('apartment')
# ds = Dataset('eth')
# ds = Dataset('gazebo_summer')
# ds = Dataset('gazebo_winter')
ds = Dataset('stairs')
ids = ds.ids[::5]

min_depth = 1.0
max_depth = 15.0
grid_res = 0.05
k = None
# k = 9
# r = None
# r = 0.15
r = 3 * grid_res

figsize = 6.4, 6.4

In [ ]:
# Functions and helpers

# Fit models dependent on incidence angle
def domain(model, n=100):
    if isinstance(model, Polynomial):
        return np.linspace(model.domain[0], model.domain[1], n)
    if isinstance(model, np.ndarray):
        return np.linspace(np.nanmin(model), np.nanmax(model), n)
    raise ValueError('Invalid domain input, only polynomial or data sample is supported.')

def lims(x):
    return np.nanquantile(x, [0.001, 0.999])

import matplotlib.pyplot as plt
# figsize = 8.27, 8.27
figsize = 6.4, 6.4

def plot_fit(x, y, x_label='x', y_label='y', x_lims=None, y_lims=None):
    if x_lims is None:
        x_lims = lims(x)
    if y_lims is None:
        y_lims = lims(y)
    poly1 = Polynomial.fit(x, y, 1).convert()
    poly2 = Polynomial.fit(x, y, 2).convert()
    print('%s to %s (deg. 1 fit): %s' % (y_label, x_label, poly1))
    # print('%s to %s (deg. 2 fit): %s' % (y_label, x_label, poly2))
    # xs = domain(poly1)
    xs = domain(x)
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.plot(x, y, '.', markersize=1, label='data')
    ax.plot(xs, poly1(xs), 'r-', linewidth=1, label='fit deg. 1')
    ax.plot(xs, poly2(xs), 'g--', linewidth=1, label='fit deg. 2')
    ax.set_xlim(x_lims)
    ax.set_ylim(y_lims)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.grid(True)
    ax.legend()
    fig.tight_layout()
    fig.show()
    # print(np.nanquantile(x, np.linspace(0.0, 1.0, 10)))
    # print(np.nanquantile(y, np.linspace(0.0, 1.0, 10)))

In [ ]:
# Depth bias estimation

for id in ids:
    t = timer()
    cloud = ds.local_cloud(id)
    pose = torch.tensor(ds.cloud_pose(id))
    dc = DepthCloud.from_points(cloud)
    # print('%i points read from dataset %s, cloud %i (%.3f s).'
    #       % (dc.size(), ds.name, id, timer() - t))

    dc = filter_depth(dc, min=min_depth, max=max_depth, log=False)

    t = timer()
    dc = filter_grid(dc, grid_res, keep='last')
    # print('%i points kept by grid filter with res. %.2f m (%.3f s).'
    #       % (dc.size(), grid_res, timer() - t))

    dc = dc.transform(pose)
    dc.update_all(k=k, r=r)
    # keep = filter_eigenvalue(dc, 0, max=(grid_res / 5)**2, only_mask=True)
    # keep = keep & filter_eigenvalue(dc, 1, min=grid_res**2, only_mask=True)
    # dc = dc[keep]
    # dc.update_all(r=r)

    clouds.append(dc)
    poses.append(pose)

dc = DepthCloud.concatenate(clouds, True)
# dc.visualize(colors='inc_angles')
# dc.visualize(colors='z')

dc.update_all(k=k, r=r)

# Visualize incidence angle to plane distance.
# TODO: Compare using plane fit for low incidence angle.
depth = dc.depth.detach().numpy().ravel()
inc = dc.inc_angles.detach().numpy().ravel()
# scaled_inc = depth * inc
inv_cos = 1.0 / np.cos(inc)
# scaled_inv_cos = depth * inv_cos
# dist = dc.normals.inner(dc.points - dc.mean)
dist = (dc.normals * (dc.points - dc.mean)).sum(dim=1).detach().numpy().ravel()
norm_dist = dist / depth

print('%s[%s]' % (ds.name, ids))

plot_fit(inc, dist,
         'Incidence Angle', 'Distance to Plane [m]')
plot_fit(inc, norm_dist,
         'Incidence Angle', 'Distance to Plane / Depth')
plot_fit(inv_cos, norm_dist,
         '1 / Incidence Angle Cosine', 'Distance to Plane / Depth',
         x_lims=[1.0, 11.47])